Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 01: Quick introduction of Cybertron

In [1]:
import sys
import time
import numpy as np
from mindspore import nn
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train.callback import LossMonitor
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
from cybertron import Cybertron
from cybertron.train import WithLabelLossCell

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
train_file = sys.path[0] + '/dataset_qm9_origin_trainset_1024.npz'
train_data = np.load(train_file)

idx = [0]  # diple
num_atom = int(train_data['num_atoms'])
scale = train_data['scale'][idx]
shift = train_data['shift'][idx]

In [3]:
net = Cybertron(model='schnet', readout='graph', dim_output=1,
                num_atoms=num_atom, length_unit='nm', energy_unit='kj/mol')
net.set_scaleshift(scale, shift)
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  SchNet
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): GaussianBasis
          Minimum distance: 0.0 nm
          Maximum distance: 1.0 nm
          Sigma for Gaussian: 0.03 nm
          Interval for Gaussian: 0.016 nm
          Number of basis functions: 64
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 64
--------------------------------------------------------------------------------
       Using 3 independent interaction layers:
--------------------------------------------------------------------------------
       0. SchNet Interaction La

In [4]:
tot_params = 0
for i, param in enumerate(net.get_parameters()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 64)
1 model.interactions.0.atomwise_bc.weight (64, 64)
2 model.interactions.0.atomwise_bc.bias (64,)
3 model.interactions.0.atomwise_ac.mlp.0.weight (64, 64)
4 model.interactions.0.atomwise_ac.mlp.0.bias (64,)
5 model.interactions.0.atomwise_ac.mlp.1.weight (64, 64)
6 model.interactions.0.atomwise_ac.mlp.1.bias (64,)
7 model.interactions.0.dis_filter.dense_layers.mlp.0.weight (64, 64)
8 model.interactions.0.dis_filter.dense_layers.mlp.0.bias (64,)
9 model.interactions.0.dis_filter.dense_layers.mlp.1.weight (64, 64)
10 model.interactions.0.dis_filter.dense_layers.mlp.1.bias (64,)
11 model.interactions.1.atomwise_bc.weight (64, 64)
12 model.interactions.1.atomwise_bc.bias (64,)
13 model.interactions.1.atomwise_ac.mlp.0.weight (64, 64)
14 model.interactions.1.atomwise_ac.mlp.0.bias (64,)
15 model.interactions.1.atomwise_ac.mlp.1.weight (64, 64)
16 model.interactions.1.atomwise_ac.mlp.1.bias (64,)
17 model.interactions.1.dis_filter.dense_layers.m

In [5]:
N_EPOCH = 8
REPEAT_TIME = 1
BATCH_SIZE = 32

ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'Z': train_data['Z'], 'E': train_data['E'][:, idx]}, shuffle=True)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_train = ds_train.repeat(REPEAT_TIME)
loss_network = WithLabelLossCell('RZE', net, nn.MAELoss())

WithLabelLossCell with input type: RZE


In [6]:
lr = 1e-3
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [7]:
model = Model(loss_network, optimizer=optim)

In [8]:
monitor_cb = LossMonitor(16)

In [9]:
outdir = 'Tutorial_C01'
params_name = outdir + '_' + net.model_name
config_ck = CheckpointConfig(
    save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(
    prefix=params_name, directory=outdir, config=config_ck)

In [10]:
print("Start training ...")
beg_time = time.time()
model.train(N_EPOCH, ds_train, callbacks=[
    monitor_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

Start training ...
epoch: 1 step: 16, loss is 0.9659242630004883
epoch: 1 step: 32, loss is 1.1600996255874634
epoch: 2 step: 16, loss is 0.9497648477554321
epoch: 2 step: 32, loss is 0.962729275226593
epoch: 3 step: 16, loss is 0.9912737607955933
epoch: 3 step: 32, loss is 0.7092443704605103
epoch: 4 step: 16, loss is 1.0484178066253662
epoch: 4 step: 32, loss is 0.8750514984130859
epoch: 5 step: 16, loss is 1.184203863143921
epoch: 5 step: 32, loss is 0.8099052906036377
epoch: 6 step: 16, loss is 1.0486085414886475
epoch: 6 step: 32, loss is 0.7762694954872131
epoch: 7 step: 16, loss is 0.8246721029281616
epoch: 7 step: 32, loss is 0.7397886514663696
epoch: 8 step: 16, loss is 0.787693977355957
epoch: 8 step: 32, loss is 0.9199405908584595
Training Fininshed!
Training Time: 00:00:14
